Install and import libraries

In [1]:
# Install necessary libraries
!pip install torch torchvision

# Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms


Define Data Transformations

In [2]:
# Define transformations for resizing, normalizing, and augmenting the dataset
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


Load CIFAR100 Dataset

In [3]:
# Define dataset root directory
dataset_root = './data'

# Load CIFAR100 dataset
trainset = datasets.CIFAR100(root=dataset_root, train=True, transform=data_transforms, download=True)
testset = datasets.CIFAR100(root=dataset_root, train=False, transform=data_transforms, download=True)


100%|██████████| 169M/169M [00:01<00:00, 104MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


Create Data Loaders

In [4]:
# Define data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)


Load Pretrained VGG16 Model

In [5]:
# Load pretrained VGG16 model
model = models.vgg16(pretrained=True)

# Modify the last fully connected layer for CIFAR100
num_in_ftrs = model.classifier[6].in_features
num_cls = 100  # CIFAR100 has 100 classes
model.classifier[6] = nn.Linear(num_in_ftrs, num_cls)

# Freeze all layers except the last layer
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier[6].parameters():
    param.requires_grad = True


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 221MB/s]


Set Device and Hyperparams

In [6]:
# Move model to GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Set the number of epochs
num_epochs = 10


Train model

In [7]:
# Training loop
best_acc = 0.0
best_model_wts = model.state_dict()

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    model.train()
    running_loss = 0.0
    running_corrects = 0

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels.data)

    scheduler.step()

    epoch_loss = running_loss / len(trainset)
    epoch_acc = running_corrects.double() / len(trainset)
    print(f'Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = model.state_dict()

torch.save(best_model_wts, 'best_model_VGG.pth')


Epoch 1/10
----------
Loss: 2.7018 Acc: 0.3811
Epoch 2/10
----------
Loss: 1.8611 Acc: 0.5123
Epoch 3/10
----------
Loss: 1.7028 Acc: 0.5384
Epoch 4/10
----------
Loss: 1.6192 Acc: 0.5537
Epoch 5/10
----------
Loss: 1.5587 Acc: 0.5672
Epoch 6/10
----------
Loss: 1.5266 Acc: 0.5723
Epoch 7/10
----------
Loss: 1.4971 Acc: 0.5808
Epoch 8/10
----------
Loss: 1.4541 Acc: 0.5895
Epoch 9/10
----------
Loss: 1.4522 Acc: 0.5945
Epoch 10/10
----------
Loss: 1.4460 Acc: 0.5938


Test Model

In [8]:
# Load the best model weights
model.load_state_dict(torch.load('best_model_VGG.pth'))
model.eval()

running_corrects = 0

for images, labels in testloader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    _, preds = torch.max(outputs, 1)
    running_corrects += torch.sum(preds == labels.data)

test_acc = running_corrects.double() / len(testset)
print(f'Test Accuracy: {test_acc:.4f}')


<ipython-input-8-4420f1249727>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_VGG.pth'))


Test Accuracy: 0.6138
